Reference
- "Numerical Methods for Engineers" Steven C. Chapra, and Raymond P. Canale

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_rows", None)

In [2]:
# goal: solve linear algebraic equations
# 1. square matrix (n equations, n unknowns)
# 2. only want the one unique solution (determinant not zero, nonsigular, invertible)

# method: Gauss elimination with partial pivoting

# partial pivoting
# 1. switch rows so that the largest element is the pivot element,
#    if equal or close to zero, end process
# 2. avoid divison by zero
# 3. minimize round-off error, serves as a remedy for ill-conditioning
#    (wide range of answers can approximately satisfy the equations)

### Problem 9.11
Given the equations

$2x_1 - 6x_2 - x_3 = -38$

$-3x_1 - x_2 + 7x_3 = -34$

$-8x_1 + x_2 - 2x_3 = -20$

(a) Solve by Gauss elimination with partial pivoting.  Show all steps of the computation.

(b) Substitute your results into the original equations to check your answers.

### Answer

1. Define Function

In [3]:
def gauss(a, b, n, tol = 0.00001):
    """
    This function implements Gauss elimination with partial pivoting
      using the pseudocode from figure 9.6 in the textbook
    
    a: array (n * n) of left-hand-side coefficients of linear equations
    b: array (1 * n) of right-hand-side constants of linear equations
    n: number of equations
    tol: parameter set by the user to a small number in order to detect near-zero occurrences
    
    Returns: the list of x variables, the result for linear equations
    """
    print('-------------Original Augmented Matrix-----------------')
    print('a|b: \n', np.concatenate((a, b.reshape(n, 1)), axis=1))
    
    x = [0] * n # contain the result of provided linear equations
    er = 0
    
    s = [0] * n # keep the largest coefficient (absolute value)
    for i in range(n):
        s[i] = abs(a[i][0])
        for j in range(1, n):
            if abs(a[i][j]) > s[i]: # keep largest coefficient, for partial pivoting
                s[i] = abs(a[i][j])
                
    print('----------------Forward Elimination Process------------------')
    er = eliminate(a, s, n, b, tol)
    # If it passes back a value of er=-1, a singular matrix has been detected
    # and the computation should be terminated
    if er == -1:
        return 'No Proper Result'
    else:
        print('----------------Backward Substitution Process------------------')
        _ = substitude(a, n, b, x) 
        
    return x

    
def eliminate(a, s, n, b, tol):
    """
    This function implements forward-elimination with partial pivoting
    """
    er = 0
    for k in range(0, n-1):
        # 1. partial pivoting (switch rows)
        # k is the row number used as pivoting in this loop
        pivot(a, b, s, n, k)
        if abs(a[k][k] / s[k]) < tol:
            er = -1
            break
        # 2. naive elimination (eliminate)
        for i in range(k + 1, n):
            # eliminate for each row
            factor = a[i][k] / a[k][k]
            for j in range(n): # modified, since the code in textbook is propabaly wrong
                # for each item in each row
                a[i][j] = a[i][j] - factor * a[k][j]
            b[i] = b[i] - factor * b[k]
        print('a|b: \n', np.concatenate((a, b.reshape(n, 1)), axis=1))
            
    if abs(a[n-1][n-1] / s[n-1]) < tol:
        er = -1
        
    return er


def pivot(a, b, s, n, k):
    """
    This function implements partial pivoting
    """
    p = k
    big = abs(a[k][k] / s[k])
    for ii in range(k+1, n):
        dummy = abs(a[ii][k] / s[ii])
        if dummy > big:
            big = dummy
            p = ii
    
    if p != k:
        for jj in range(k, n):
            a[p][jj], a[k][jj] = a[k][jj], a[p][jj]
        b[p], b[k] = b[k], b[p]
        s[p], s[k] = s[k], s[p]
        
    return None


def substitude(a, n, b, x):
    """
    This function implements backward-substitution
    """
    x[n-1] = b[n-1] / a[n-1][n-1]
    for i in range(n - 2, -1, -1):
        sum_ = 0
        for j in range(i + 1, n):
            sum_ += a[i][j] * x[j]
        x[i] = (b[i] - sum_) / a[i][i] # modified, since the code in textbook is propabaly wrong
        print('x: ', x)
        
    return None

2. Implement Calculation

In [4]:
a = np.array([[2, -6, -1], [-3, -1, 7], [-8, 1, -2]]).astype(float)
b = np.array([-38, -34, -20]).astype(float)
n = 3
x = gauss(a.copy(), b.copy(), n)
print('Answer: ', x)

-------------Original Augmented Matrix-----------------
a|b: 
 [[  2.  -6.  -1. -38.]
 [ -3.  -1.   7. -34.]
 [ -8.   1.  -2. -20.]]
----------------Forward Elimination Process------------------
a|b: 
 [[ -8.      1.     -2.    -20.   ]
 [  0.     -1.375   7.75  -26.5  ]
 [  0.     -5.75   -1.5   -43.   ]]
a|b: 
 [[ -8.           1.          -2.         -20.        ]
 [  0.          -5.75        -1.5        -43.        ]
 [  0.           0.           8.10869565 -16.2173913 ]]
----------------Backward Substitution Process------------------
x:  [0, 8.0, -2.0]
x:  [4.0, 8.0, -2.0]
Answer:  [4.0, 8.0, -2.0]


3. Check Answer

In [5]:
# 1. substitute the results into original equations
print('differences: ', b - np.dot(a, x))

differences:  [0. 0. 0.]


In [6]:
# 2. use the numpy library
x = np.linalg.solve(a, b)
x

array([ 4.,  8., -2.])

### Problem 9.18
Develop, debug, and test a program in either a high-level language or macro language of your choice to solve a system of equations with Gauss elimination with partial pivoting.  Base the program on the pseudocode from Fig.9.6.  Test the program using the following system (which has an answer of $x_1 = x_2 = x_3 = 2$),

$x_1 + 2x_2 - x_3 = 4$

$5x_1 + 2x_2 + 2x_3 = 18$

$-3x_1 + 5x_2 - x_3 = 2$

### Answer

1. Implement Calculation

In [7]:
a = np.array([[1, 2, -1], [5, 2, 2], [-3, 5, -1]]).astype(float)
b = np.array([4, 18, 2]).astype(float)
n = 3
x = gauss(a.copy(), b.copy(), n)
print('Answer: ', x)

-------------Original Augmented Matrix-----------------
a|b: 
 [[ 1.  2. -1.  4.]
 [ 5.  2.  2. 18.]
 [-3.  5. -1.  2.]]
----------------Forward Elimination Process------------------
a|b: 
 [[ 5.   2.   2.  18. ]
 [ 0.   1.6 -1.4  0.4]
 [ 0.   6.2  0.2 12.8]]
a|b: 
 [[ 5.          2.          2.         18.        ]
 [ 0.          6.2         0.2        12.8       ]
 [ 0.          0.         -1.4516129  -2.90322581]]
----------------Backward Substitution Process------------------
x:  [0, 1.9999999999999998, 2.0]
x:  [2.0, 1.9999999999999998, 2.0]
Answer:  [2.0, 1.9999999999999998, 2.0]


2. Check Answer

In [8]:
# 1. substitute the results into original equations
print('differences: ', b - np.dot(a, x))

differences:  [0.00000000e+00 0.00000000e+00 1.77635684e-15]


In [9]:
# 2. use the numpy library
x = np.linalg.solve(a, b)
x

array([2., 2., 2.])